In [2]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd

# Adjust if your .env lives elsewhere
load_dotenv()  # loads .env in current working dir (or parent if you launch from repo root)

db_url = os.getenv("DATABASE_URL")
assert db_url, "DATABASE_URL is not set. Add it to .env or set env var."

engine = create_engine(db_url)
print("✅ Connected (engine created)")

✅ Connected (engine created)


In [3]:
tables = [
    "tennis_players",
    "tennis_matches",
    "tennis_player_snapshots",
    "tennis_match_stats_ta",
    "tennis_features_ta",
]

counts = []
with engine.begin() as conn:
    for t in tables:
        try:
            n = conn.execute(text(f"SELECT COUNT(*) FROM {t}")).scalar_one()
            counts.append({"table": t, "rows": n})
        except Exception as e:
            counts.append({"table": t, "rows": None, "error": str(e)})

pd.DataFrame(counts)


,table,rows
0,tennis_players,132887
1,tennis_matches,25141
2,tennis_player_snapshots,0
3,tennis_match_stats_ta,24090
4,tennis_features_ta,24090


In [4]:
player_name = "Carlos Alcaraz"  # change me

with engine.begin() as conn:
    df = pd.read_sql(text("""
        SELECT id, name, gender, ta_player_id, ta_slug
        FROM tennis_players
        WHERE name ILIKE :q
        ORDER BY gender, id
        LIMIT 20
    """), conn, params={"q": f"%{player_name}%"})

df


,id,name,gender,ta_player_id,ta_slug
0,25639,Carlos Alcaraz Gonzalez,M,106688,m-106688-carlos-alcaraz-gonzalez
1,79200,Carlos Alcaraz,M,207989,m-207989-carlos-alcaraz


In [5]:
player_name = "Carlos Alcaraz"  # change me
limit = 20

with engine.begin() as conn:
    df = pd.read_sql(text("""
        SELECT match_date, tour, tournament, surface, round, status, p1_name, p2_name, score
        FROM tennis_matches
        WHERE p1_name = :name OR p2_name = :name
        ORDER BY match_date DESC
        LIMIT :lim
    """), conn, params={"name": player_name, "lim": limit})

df

,match_date,tour,tournament,surface,round,status,p1_name,p2_name,score
0,2024-11-19,ATP,Davis Cup Finals QF: NED vs ESP,Hard,RR,finished,Carlos Alcaraz,Tallon Griekspoor,7-6(0) 6-3
1,2024-11-11,ATP,Tour Finals,Hard,RR,finished,Alexander Zverev,Carlos Alcaraz,7-6(5) 6-4
2,2024-11-11,ATP,Tour Finals,Hard,RR,finished,Casper Ruud,Carlos Alcaraz,6-1 7-5
3,2024-11-11,ATP,Tour Finals,Hard,RR,finished,Carlos Alcaraz,Andrey Rublev,6-3 7-6(8)
4,2024-10-28,ATP,Paris Masters,Hard,R32,finished,Carlos Alcaraz,Nicolas Jarry,7-5 6-1
5,2024-10-28,ATP,Paris Masters,Hard,R16,finished,Ugo Humbert,Carlos Alcaraz,6-1 3-6 7-5
6,2024-10-02,ATP,Shanghai Masters,Hard,QF,finished,Tomas Machac,Carlos Alcaraz,7-6(5) 7-5
7,2024-10-02,ATP,Shanghai Masters,Hard,R16,finished,Carlos Alcaraz,Gael Monfils,6-4 7-5
8,2024-10-02,ATP,Shanghai Masters,Hard,R32,finished,Carlos Alcaraz,Yibing Wu,7-6(5) 6-3
9,2024-10-02,ATP,Shanghai Masters,Hard,R64,finished,Carlos Alcaraz,Juncheng Shang,6-2 6-2


In [6]:
player_id = 123  # <-- put the real id here

with engine.begin() as conn:
    snap_counts = pd.read_sql(text("""
        SELECT surface, COUNT(*) AS rows, MAX(as_of) AS latest_as_of
        FROM tennis_player_snapshots
        WHERE player_id = :pid
        GROUP BY surface
        ORDER BY rows DESC
    """), conn, params={"pid": player_id})

snap_counts


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "as_of" does not exist
LINE 2:         SELECT surface, COUNT(*) AS rows, MAX(as_of) AS late...
                                                      ^

[SQL: 
        SELECT surface, COUNT(*) AS rows, MAX(as_of) AS latest_as_of
        FROM tennis_player_snapshots
        WHERE player_id = %(pid)s
        GROUP BY surface
        ORDER BY rows DESC
    ]
[parameters: {'pid': 123}]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
with engine.begin() as conn:
    sample = pd.read_sql(text("""
        SELECT *
        FROM tennis_match_stats_ta
        ORDER BY match_date DESC
        LIMIT 5
    """), conn)

sample
